# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from z3 import *
from itertools import combinations
import sys
sys.path.append('../../scripts/shatter/')
from SHATTER_Control import *
import warnings
warnings.filterwarnings('ignore')

# Fixed Parameter

In [2]:
CO2_FRESH_AIR = 400             # CO2 concentration (ppm) of fresh air
TEMP_FRESH_AIR = 91.4           # Temperature (33 degree F) of the fresh air
CP_AIR = 1.026                  # Specific heat of fresh air
DEF_TEMP_SUPPLY_AIR =  55.4     # Default temperature (degree fahrenheit) of supply air (13 degree celsius)
MINUTES_IN_A_DAY = 1440         # Number of minutes in a day
OFF_PEAK_ENERGY_COST = 0.34     # OFF-PEAK energy cost (USD)
ON_PEAK_ENERGY_COST = 0.48      # ON-PEAK energy cost (USD)
ON_PEAK_START_SLOT = 960        # ON-PEAK start time (minute in a day)
ON_PEAK_END_SLOT = 1260         # ON-PEAK end time (minute in a day)
BATTER_STORAGE = 0.48           # Energy (kWh) produced by battery

# Variable Parameters

In [3]:
zone_temp_setpoint = [0, 75.2, 75.2, 75.2, 75.2]     # list of temperature (fahrenheit) setpoint of the different zones
zone_co2_setpoint = [0, 1000, 1000, 1000, 1000]      # list of CO2 (ppm) setpoint of the corresponding zones
control_time = 1                                     # control time (in minute)

# Accessing Zone-Activity-Appliances Information

In [4]:
zones = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Zone-Info')
zone_volume = zones["Volume (cf)"].to_list()                  # Zones' volumes (cubic feet)     
        
activities = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Activity-Info')
energy_appliances = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Appliances-Info').iloc[:,-1].to_numpy()

num_activities = len(activities)  

activity_zone_map = dict()
for i in range(num_activities):
    activity_zone_map[int(activities["Activity ID"][i])] = int(activities["Zone ID"][i])

benign_activity_appliances_map = pd.read_excel('../../data/Aras-Information.xlsx', sheet_name='Benign-Activity-Appliances-Map').iloc[:, 4:].to_numpy()
  
activity_zone_temp_setpoint = [0]
activity_zone_co2_setpoint = [0] 
activity_pp_co2 = [0] + activities['CO2 Emission by Occupant (CFM)'].to_list()    # CO2 Emission by Occupant (cfm)
activity_pp_heat = [0] + activities["Heat Radiation by Occupant (W)"].to_list()   # Heat Radiation by Occupant (W)
activity_load = [0]                                                               # Heat radiated by Appliances (W)
activity_zone_volume = [0]

for i in range(1, num_activities + 1):
    activity_zone_temp_setpoint.append(zone_temp_setpoint[activity_zone_map[i]])
    activity_zone_co2_setpoint.append(zone_co2_setpoint[activity_zone_map[i]])
    activity_zone_volume.append(zone_volume[activity_zone_map[i]])
    activity_load.append(sum(benign_activity_appliances_map[i] * energy_appliances))

# Generating Possible Combination of Occupants

In [5]:
activities = [i for i in range(1, num_activities + 1)]

indexes = list(combinations(activities, 2))

unique_samples = np.zeros((len(indexes) + num_activities, num_activities + 1))

for i in range(len(indexes)):
    unique_samples[i][indexes[i][0]] = 1
    unique_samples[i][indexes[i][1]] = 1

count = 1
for i in range(len(indexes), len(indexes) + num_activities):
    unique_samples[i][count] = 2
    count += 1

# Tabulate the Control Costs for all Possible Combinations

In [6]:
# convert list to a tuple
def to_tuple(a):
    try:
        return tuple(to_tuple(i) for i in a)
    except TypeError:
        return a
    
dict_control_cost = dict()
for i in range(len(unique_samples)):
    print(i)
    sample = unique_samples[i]
    activity_zone_occupant = sample.tolist()
    dict_control_cost[to_tuple(sample)] = shatter_control_cost(activity_zone_occupant, activity_zone_temp_setpoint, activity_zone_volume, activity_pp_co2, activity_pp_heat, activity_load, activity_zone_co2_setpoint, control_time)
dict_control_cost

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{(0.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.0,
 (0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.09156150442045,
 (0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.007328273316,
 (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0): 0.09156150442045,
 (0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
 

# SHATTER Control Cost Calculation Functions

In [7]:
def shatter_control(house, filename):
    control_costs = []
    unit_energy_costs = []
    energy_consumptions = []
    dataset = pd.read_csv('../../data/processed/Processed-Dataframe_House-' + house + '_SHATTER.csv')
    
    prev_day = -1
    battery_usage = 0 # in kWh
    
    for i in range(len(dataset)):
        
        current_day = int(dataset.iloc[i,0])
        current_minute = int(dataset.iloc[i,1])
        
        if current_day!= prev_day:
            battery_usage = 0
            prev_day = current_day        
            print(house, current_day)
        
        activity_zone_occupants = [0] * (num_activities + 1)
        occ_1_activity_id = dataset["Occupant-1 Activity"][i]
        occ_2_activity_id = dataset["Occupant-2 Activity"][i]
        activity_zone_occupants[occ_1_activity_id] += 1
        activity_zone_occupants[occ_2_activity_id] += 1
        
        energy_consumption = dict_control_cost[to_tuple(activity_zone_occupants)]
        
        if current_minute < ON_PEAK_START_SLOT or current_minute > ON_PEAK_END_SLOT:
            unit_energy_cost = OFF_PEAK_ENERGY_COST
        else:
            if battery_usage < BATTER_STORAGE:
                battery_usage += energy_consumption
                unit_energy_cost = OFF_PEAK_ENERGY_COST
            else:
                unit_energy_cost = ON_PEAK_ENERGY_COST
                
        current_cost = energy_consumption * unit_energy_cost
        control_costs.append(current_cost)
        unit_energy_costs.append(unit_energy_cost)
        energy_consumptions.append(energy_consumption)
        
        
    dataset['Unit Energy Cost ($)'] = unit_energy_costs
    dataset['Energy Comsumption (kWh)'] = energy_consumptions
    dataset['Control Cost ($)'] = control_costs
    
    dataset.to_csv(filename, index = False)

# Saving Datasets

In [8]:
shatter_control("A", "../../data/shatter/SHATTER-Control-Dataframe_House-A.csv")
shatter_control("B", "../../data/shatter/SHATTER-Control-Dataframe_House-B.csv")

A 1
A 2
A 3
A 4
A 5
A 6
A 7
A 8
A 9
A 10
A 11
A 12
A 13
A 14
A 15
A 16
A 17
A 18
A 19
A 20
A 21
A 22
A 23
A 24
A 25
A 26
A 27
A 28
A 29
B 1
B 2
B 3
B 4
B 5
B 6
B 7
B 8
B 9
B 10
B 11
B 12
B 13
B 14
B 15
B 16
B 17
B 18
B 19
B 20
B 21
B 22
B 23
B 24
B 25
B 26
B 27
B 28
B 29
